## RQ2 part1

In [53]:
from collections import defaultdict
import numpy as np
import pickle
import networkx as nx

In [54]:
#we create two lists edges and n, one for edges and the last to save al nodes that we find in edges
edges = []
n =[]
#we import data that we need and we use them to fill previous initialized lists 
with open("wiki-topcats-reduced.txt\wiki-topcats-reduced.txt") as mytxt:
    for line in mytxt:
        #splitting edge to get the two nodes that form it
        i,j= line.split()
        #adding the nodes to n list
        n.extend((int(i),int(j)))
        #adding the edge to edges list
        edges.append([int(i),int(j)])

In [2]:
#we eliminate duplicates from nodes
nodes = set(n)

We create a dictionary with articles for each category, we take only articles in the reduced version of the graph 
and only categories with more than 3500 articles.

In [3]:
#creating the dictionary
cat_dic = {}
with open ("wiki-topcats-categories.txt")as dc:
    for line in dc:
        #initializing a set to save articles of categories that are in the graph
        art1 = set([])
        #eliminating from the line the unuseful part
        l1 = line.replace("Category:","")
        #splitting name of the category from its articles
        l2 = l1.split(";")
        k = l2[0]
        #converting articles in int
        art = list(map(int,l2[1].split()))
        #checking if articles are in our graph
        for a in art :
            if a in nodes:
                art1.add(a) 
        #checking if the category meets the requirement and saving it into the dictionary
        if len(art1)>3500:
             cat_dic[k] = art1

We store the graph in a dictionary saving from each source node a list with end nodes of its edges
we use defaultdict to have a list as a default attribute.

In [4]:
g = defaultdict(list)
for e in edges:
    g[e[0]].append(e[1])
    

With this algorithm, derived in part from bfs,we add the distance(of the shortest path) from the starting point to each 
node that is connected to the starting point, we save thtis data in a dictionary. The function requires a starting graph in dictionary format,a starting point and the dictionary to add distances.

In [ ]:
def bfs_mod(graph, start,dists):
    #initialize a set with visited
    visited = set([])
    #we begin the queue with the starting point
    queue = [start]
    #the counter,it starts from 0
    c = 0
    while queue:
        l =[]
        #for each node at this level wee add the distance from the starting point
        for q in queue:
            if q not in visited:
                #appending the distance of the shortest path to dists for the node(q) key
                dists[q].append(c)
                #adding q to visited
                visited.add(q)
                #adding successors of q to the queue
                l.extend(graph[q])
        queue = l
        #we increase the counter(distance)
        c+=1

Here we define a dictionary with all nodes in the graph as keys and we initialize a list as attribute to append 
distances.

In [6]:
dists = {n:[] for n in nodes}

We execute the bfs_mod for each node of the input category, so we'll have in dists for each node a list with all lengths
of  the shortest path  to that node having as starting node a node from the input category('Association_football_goalkeepers').

In [7]:
for article in cat_dic['Association_football_goalkeepers']:
    bfs_mod(g,article,dists)


Here we check if in some cases there was no link between one node of the input categories and one of the other 
categories, we simply add 100(a number greater of the found shortest paths lengths) to the list of distances 
in that case. With this procedure we don't keep track of what source node wasn't connected to the target node, 
but since our goal is the reasearch of the median this information is not important. This is done to have an effect on the calculation of the median and so missing paths will have a crucial role to determine the ranking block.

In [9]:
for article in dists.keys():
    #searching if in some cases there was no path comparing the expected length of of the list with the real length
    if len(dists[article])<len(cat_dic["Association_football_goalkeepers"]):
                    #adding 100 for each mssing value to the list of distanes of the article
                    l1=[100]*(len(cat_dic['Association_football_goalkeepers'])-len(dists[article]))
                    dists[article].extend(l1)
           

Finally we calculate the distance of each category from the input category using the median of the distances of the shortest paths

In [18]:
#we create a list medians where we'll save the block ranking with the value of the median for each block
medians = []
for k in cat_dic.keys():
    if k != 'Association_football_goalkeepers':
            d = []
            #retrieving distances for each article of the category
            for article in cat_dic[k]:
                  d.extend(dists[article])
            #calculating the median
            med = np.median(d)
            #finally appending the values to medians
            medians.append([k,med])

In [42]:
#showing unordered medians
medians

[['Association_football_goalkeepers', 0],
 ['British_films', 6.0],
 ['English-language_films', 6.0],
 ['American_films', 6.0],
 ['American_television_actors', 6.0],
 ['American_film_actors', 6.0],
 ['English_footballers', 7.0],
 ['The_Football_League_players', 7.0],
 ['Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 7.0],
 ['Indian_films', 7.0],
 ['Rivers_of_Romania', 7.0],
 ['English-language_albums', 7.0],
 ['People_from_New_York_City', 7.0],
 ['Debut_albums', 7.0],
 ['Black-and-white_films', 7.0],
 ['American_military_personnel_of_World_War_II', 7.0],
 ['Association_football_forwards', 8.0],
 ['Harvard_University_alumni', 8.0],
 ['Major_League_Baseball_pitchers', 8.0],
 ['Year_of_birth_missing_(living_people)', 8.0],
 ['Place_of_birth_missing_(living_people)', 8.0],
 ['Windows_games', 8.0],
 ['Association_football_midfielders', 100.0],
 ['Association_football_defenders', 100.0],
 ['Year_of_death_missing', 100.0],
 ['Main_Belt_asteroids', 100.0],
 ['Asteroids_na

In [43]:
#we insert in position 0 of the ranking block the input category with median value 0
medians.insert(0,['Association_football_goalkeepers',0])

In [44]:
#we sort the block according to the median since medians is unordered
medians.sort(key = lambda x : x[1])

In [47]:
# creating and showing the ranking block without medians values
ranking_block= []
for m in medians:
    ranking_block.append(m[0])
ranking_block

['Association_football_goalkeepers',
 'British_films',
 'English-language_films',
 'American_films',
 'American_television_actors',
 'American_film_actors',
 'English_footballers',
 'The_Football_League_players',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Indian_films',
 'Rivers_of_Romania',
 'English-language_albums',
 'People_from_New_York_City',
 'Debut_albums',
 'Black-and-white_films',
 'American_military_personnel_of_World_War_II',
 'Association_football_forwards',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'Year_of_birth_missing_(living_people)',
 'Place_of_birth_missing_(living_people)',
 'Windows_games',
 'Living_people',
 'Association_football_midfielders',
 'Association_football_defenders',
 'Year_of_death_missing',
 'Main_Belt_asteroids',
 'Asteroids_named_for_people',
 'Year_of_birth_missing']

In [48]:
#saving medians with pickle
with open('medians', 'wb') as f:
        pickle.dump(medians, f)

In [16]:
#saving dists with pickle
with open('dists.pickle', 'wb') as f:
    pickle.dump(dists, f, protocol=pickle.HIGHEST_PROTOCOL)

## RQ2 part 2

First two cells are naturally unnecessary if we run them before in the first part, but we put them here also for completeness

In [5]:
#we create two lists edges and n, one for edges and the last to save al nodes that we find in edges
edges = []
n =[]
#we import data that we need and we use them to fill previous initialized lists 
with open("wiki-topcats-reduced.txt\wiki-topcats-reduced.txt") as mytxt:
    for line in mytxt:
        #splitting edge to get the two nodes that form it
        i,j= line.split()
        #adding the nodes to n list
        n.extend((int(i),int(j)))
        #adding the edge to edges list
        edges.append([int(i),int(j)])
#removing duplicates from n
nodes = set(n)

In [41]:
#we create a dictionary with articles for each category, we take only articles in the reduced version of the graph 
#and only categories with more than 3500 articles
cat_dic = {}
with open ("wiki-topcats-categories.txt")as dc:
    for line in dc:
        #initializing set to save articles of categories that are in the graph
        art1 = set([])
        #eliminating from the line  the unuseful part
        l1 = line.replace("Category:","")
        #splitting name of the category from its articles and trnsforming them in int
        l2 = l1.split(";")
        k = l2[0]
        art = list(map(int,l2[1].split()))
        #checking if articles are in our graph
        for i in art :
            if i in nodes:
                art1.add(i) 
        #checking if the category meets the requirement and saving it into the dictionary
        if len(art1)>3500:
             cat_dic[k] = art1

We eliminate duplicates articles from categories that are in a more distant position from the input category


In [43]:
#initalizing assigned set , used to keep track of yet assigned nodes
assigned = set([])
for category in medians:
    #initialising a list to save not already added articles for the "c" category
    usab_art = []
    for article in cat_dic[category[0]]:
        if article not in assigned:
            # for each article of the category,if needed, adding it to usab_art and assigned
            usab_art.append(article)
            assigned.add(article)
    #updating cat_dic
    cat_dic[category[0]]= usab_art

In [7]:
#we create a new G graph we networkx
G=nx.DiGraph()
#we add nodes and edges to the graph
G.add_nodes_from(nodes)
G.add_edges_from(edges)        
        

We create a dictionary to save the scores, keys of this dictionary will be the categories and this will be a nested
dictionary.

In [45]:
cat_w ={}

For each category we create a subgraph with its nodes,and we save them in a dictionary that we store in cat_w, this are not already the scores that we need because they are simply the sum of the in edges form the nodes of the same category(we use the length of the in edges, found using in_edges function from networkX). We need to add also the  scores of in_edges from the precedent category.


In [46]:
for category in medians:
    #creating subgraph
    s_g = G.subgraph(cat_dic[category[0]])
    #temporary dictionary to store results
    d={}
    #adding 1 for each in edge of the same category
    for node in s_g:
         d[node]=len(s_g.in_edges(node))
    cat_w[c[0]]=d

Here we add to score of the single node the weights of the predecessors if they are in the previous categgory, to do that we use the predecessors function from networkX.

In [47]:
#updating scores using the order of the ranking_block,starting from the second category
for i in range(1,len(medians)):
    for article in cat_w[medians[i][0]].keys():
        #creating predecessors
        predecessors = G.predecessors(article)
        for predecessor in predecessors:
            #if we find a predecessor in the previous category we add its score to the article weight to have 
            #the final score
            if p in cat_w[medians[i-1][0]].keys():
                cat_w[medians[i][0]][article]= cat_w[medians[i][0]][article]+cat_w[medians[i-1][0]][predecessor]

Finally we create a list from articles ids(ord_list), retrieving from the dictionary cat_w the scores and the articles for each category, then we sort them and we add them to the list, we do this using the order of medians, so we have the ranking block's order.

In [48]:
#initializing the list where to insert all ordered articles 
ord_list =[]
#for every category in the ranking block(we use medians as before)
for category in medians:
    #initializing a list
    l =[]
    for article in cat_w[category[0]]:
        #appending to l a list with articole and its score
        l.append([article, cat_w[category[0]][article]])
    #sorting the list    
    l.sort(key = lambda x :x[1], reverse=True)
    #adding to the final list of all articles
    ord_list.extend(l)

In [52]:
#we save ord_list with pickle
with open(r"ordinated_articles.pickle", "wb") as ord_:
           pickle.dump(ord_list, ord_)

In [50]:
#printing the first 100 elements of the list as example
for i in range(100):
    print(ord_list[i][0])

81952
81854
88889
89871
82053
82589
83527
82477
82631
83514
82736
83722
83985
81945
82422
82049
82621
82758
89712
81953
88951
81896
83536
88866
82541
82832
84393
86080
1005156
1005198
81297
81300
82598
82771
87713
87719
1358603
88993
83250
1358682
88948
82620
107564
1361043
83753
88636
88986
88987
81616
81732
82380
83500
83540
84019
86225
88884
88933
88972
89984
737907
82763
84237
895705
86410
88325
88794
81000
81097
81104
81519
81745
81764
82468
82732
82824
82910
83720
83848
84005
86520
1381405
87761
88690
88748
88897
88975
1358768
1358805
97305
81321
81456
89699
81578
82793
82831
82933
83067
83422
83596
83606
